Para la obtención de la tabla de ratios financieros, la cual es la que más trabajo conlleva, he decidido crear un Jupyter nuevo. 

Los ratios que quiero sacar son: 

-PER: Price Earnings Rate: Ratio de valoración bursatil

    Precio actual/Beneficio por acción o EPS. 
    Mide cuanto paga el mercado por cada euro de beneficio
    
    
-ROE Return on Equity: Ratio de rentabilidad

    Beneficio neto/Patrimonio Neto
    Mide la rentabilidad del capital invertido por los accionistas

-EV/EBIT: Ratio de valoración financiera

    (Market Cap - Caja + Deuda)/EBITDA
    Muestra el valor de la compañía sobre los recursos que genera sin tener en cuenta su estructura financiera

-Beta: Ratio de riesgo

    Volatilidad relativa de la acción frente al mercado.
    Covarianza(Rendimiento acción,Rendimiento mercado)/Varianza(rendimiento mercado)

-Deuda financiera/EBITDA:

    (Total Pasivo - proveedores)/EBITDA
    Ratio de endeudamiento de la empresa


In [6]:
#Importamos lo necesario para trabajar con la API Profit.com y seteamos 
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime
import time
import ast
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [18]:
#Accedo también al archivo .env para cargar mis credenciales de la API de Profit.com
import os
from dotenv import load_dotenv

# Ruta al archivo tokens.env (asumiendo que está en la misma carpeta que el notebook)
load_dotenv("tokens.env")

# Acceder a tus tokens
api_key = os.getenv("API_KEY")
otro_token = os.getenv("API_KEY_2")  # cambia el nombre según tus claves reales

print("API Key cargada:", api_key is not None)


API Key cargada: True


In [12]:
#El primer ratio que queremos obtener es el PER (P/E ratio). Para ello sacaremos una tabla con los precios de las acciones
#llamamos al archivo 03.cotizaciones_IBEX35 de la carpeta data
df_precios = pd.read_csv('data/03.cotizaciones_IBEX35.csv')
df_precios

,ticker,Date,Close,Volume,Variación,Año
0,GRF.MC,2020-01-02 00:00:00+01:00,30.99,392672,NaN,2020
1,GRF.MC,2020-01-03 00:00:00+01:00,31.02,641127,0.0009,2020
2,GRF.MC,2020-01-06 00:00:00+01:00,31.01,386910,-0.0003,2020
3,GRF.MC,2020-01-07 00:00:00+01:00,31.86,2090693,0.0274,2020
4,GRF.MC,2020-01-08 00:00:00+01:00,31.83,609008,-0.0009,2020
...,...,...,...,...,...,...
47640,PUIG.MC,2025-04-25 00:00:00+02:00,16.28,484878,0.0106,2025
47641,PUIG.MC,2025-04-28 00:00:00+02:00,16.56,555283,0.0172,2025
47642,PUIG.MC,2025-04-29 00:00:00+02:00,16.69,878013,0.0079,2025
47643,PUIG.MC,2025-04-30 00:00:00+02:00,16.43,404284,-0.0156,2025


In [13]:
#Crearemos un data frame nuevo con la última cotización de cada acción por año y ticher
df_precios_finales = df_precios.sort_values("Date").groupby(["ticker","Año"]).last().reset_index()
#eliminamos las columnas volumen y variación ya que no las necesitamos y renombreamos la columna Close a Ultimo Precio
df_precios_finales = df_precios_finales.drop(columns=["Volume","Variación"])
df_precios_finales = df_precios_finales.rename(columns={"Close":"Ultimo Precio"})
#eliminamos la datos del ticker IBEX35 ya que no nos interesa, ya que no es una acción
df_precios_finales = df_precios_finales[df_precios_finales["ticker"] != "^IBEX"]
#guardamos el dataframe en un csv para poder usarlo en el futuro en la carpeta data_calculo_ratios
df_precios_finales.to_csv('data_calculo_ratios/01.PER.cotizaciones_IBEX35_ultimos_precios.csv',index=False)
df_precios_finales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 0 to 204
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ticker         205 non-null    object 
 1   Año            205 non-null    int64  
 2   Date           205 non-null    object 
 3   Ultimo Precio  205 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 8.0+ KB


In [14]:
df_precios_finales.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ticker,205,35,ACS.MC,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Año,205.0,NaN,NaN,NaN,2022.531707,1.710756,2020.0,2021.0,2023.0,2024.0,2025.0
Date,205,6,2024-12-31 00:00:00+01:00,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ultimo Precio,205.0,NaN,NaN,NaN,23.877512,34.20967,0.29,5.18,13.59,25.58,228.2


In [15]:
df_precios_finales["ticker"].unique()   

array(['ACS.MC', 'ACX.MC', 'AENA.MC', 'AMS.MC', 'ANA.MC', 'ANE.MC',
       'BBVA.MC', 'BKT.MC', 'CABK.MC', 'CLNX.MC', 'COL.MC', 'ELE.MC',
       'ENG.MC', 'FDR.MC', 'FER.MC', 'GRF.MC', 'IAG.MC', 'IBE.MC',
       'IDR.MC', 'ITX.MC', 'LOG.MC', 'MAP.MC', 'MRL.MC', 'MTS.MC',
       'NTGY.MC', 'PUIG.MC', 'RED.MC', 'REP.MC', 'ROVI.MC', 'SAB.MC',
       'SAN.MC', 'SCYR.MC', 'SLR.MC', 'TEF.MC', 'UNI.MC'], dtype=object)

In [16]:
#generamos una lista con los tickers de las acciones que tenemos en el dataframe
tickers = df_precios_finales["ticker"].unique().tolist()
tickers


['ACS.MC',
 'ACX.MC',
 'AENA.MC',
 'AMS.MC',
 'ANA.MC',
 'ANE.MC',
 'BBVA.MC',
 'BKT.MC',
 'CABK.MC',
 'CLNX.MC',
 'COL.MC',
 'ELE.MC',
 'ENG.MC',
 'FDR.MC',
 'FER.MC',
 'GRF.MC',
 'IAG.MC',
 'IBE.MC',
 'IDR.MC',
 'ITX.MC',
 'LOG.MC',
 'MAP.MC',
 'MRL.MC',
 'MTS.MC',
 'NTGY.MC',
 'PUIG.MC',
 'RED.MC',
 'REP.MC',
 'ROVI.MC',
 'SAB.MC',
 'SAN.MC',
 'SCYR.MC',
 'SLR.MC',
 'TEF.MC',
 'UNI.MC']

In [26]:
#Sacamos usando la API de Profit.com los informes de PyG de una empresa como ejemplo. estos datos los usaremos para calcular el PER y otras ratios
#En este caso usaremos la empresa 'MRL.MC' como ejemplo.
ticker = 'MRL.MC'
URL = f"https://api.profit.com/data-api/fundamentals/stocks/income_statement/{ticker}?token=" + api_key
URL

'https://api.profit.com/data-api/fundamentals/stocks/income_statement/MRL.MC?token=978336172b8c4b29960d9366896f6ef7'

In [22]:
response = requests.get(URL)
if response.status_code == 200:
    data_MRL = response.json()
else:
    print(f"Error: {response.status_code} - {response.text}")

data_MRL

{'currency': 'EUR',
 'quarterly': {'2014-03-31': {'totalRevenue': 22850000.0,
   'costOfRevenue': None,
   'grossProfit': 22850000.0,
   'totalOperatingExpenses': 10564000.0,
   'researchDevelopment': None,
   'sellingGeneralAdministrative': None,
   'discontinuedOperations': None,
   'otherOperatingExpenses': 10564000.0,
   'operatingIncome': 12286000.0,
   'interestIncome': None,
   'totalOtherIncomeExpenseNet': -5953000.0,
   'nonOperatingIncomeNetOther': None,
   'incomeBeforeTax': 6333000.0,
   'taxProvision': None,
   'incomeTaxExpense': 536000.0,
   'extraordinaryItems': None,
   'minorityInterest': None,
   'effectOfAccountingCharges': None,
   'nonRecurring': None,
   'otherItems': None,
   'netIncome': 5797000.0,
   'preferredStockAndOtherAdjustments': None,
   'netIncomeApplicableToCommonShares': None,
   'ebit': 21329000.0,
   'dilutedWeightedAverageShares': 170.2914,
   'netIncomeAfterTaxes': None,
   'netIncomeBeforeExtraordinaryItems': None,
   'dilutedNetIncome': None,


In [37]:
#Convertimos el json a un dataframe para poder trabajar con él
df_MRL = pd.DataFrame(data_MRL).reset_index()
#renombramos la columna index a 'Año' y eliminamos la columna 'index'
df_MRL = df_MRL.rename(columns={"index":"fecha"})
df_MRL["fecha"] = pd.to_datetime(df_MRL["fecha"])
df_MRL["ticker"] = ticker
df_MRL

,fecha,currency,quarterly,yearly,ticker
0,2014-03-31,EUR,"{'totalRevenue': 22850000.0, 'costOfRevenue': None, 'grossProfit': 22850000.0, 'totalOperatingExpenses': 10564000.0, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': 10564000.0, 'operatingIncome': 12286000.0, 'interestIncome': None, 'totalOtherIncomeExpenseNet': -5953000.0, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': 6333000.0, 'taxProvision': None, 'incomeTaxExpense': 536000.0, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': 5797000.0, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': 21329000.0, 'dilutedWeightedAverageShares': 170.2914, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}",NaN,MRL.MC
1,2014-06-30,EUR,"{'totalRevenue': None, 'costOfRevenue': None, 'grossProfit': None, 'totalOperatingExpenses': 35000.0, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': 35000.0, 'operatingIncome': -35000.0, 'interestIncome': None, 'totalOtherIncomeExpenseNet': None, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': -35000.0, 'taxProvision': None, 'incomeTaxExpense': None, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': -35000.0, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': None, 'dilutedWeightedAverageShares': 141.9208, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}",NaN,MRL.MC
2,2014-09-30,EUR,"{'totalRevenue': 25873000.0, 'costOfRevenue': None, 'grossProfit': 25873000.0, 'totalOperatingExpenses': 25873000.0, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': 25873000.0, 'operatingIncome': 23734000.0, 'interestIncome': None, 'totalOtherIncomeExpenseNet': 5383000.0, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': 29117000.0, 'taxProvision': None, 'incomeTaxExpense': 21000.0, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': 29096000.0, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': None, 'dilutedWeightedAverageShares': 141.9208, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}",NaN,MRL.MC
3,2014-12-31,EUR,"{'totalRevenue': 30743000.0, 'costOfRevenue': 4553000.0, 'grossProfit': 26190000.0, 'totalOperatingExpenses': 11343000.0, 'researchDevelopment': None, 'sellingGeneralAdministrative': 18000.0, 'discontinuedOperations': None, 'otherOperatingExpenses': 15896000.0, 'operatingIncome': 14847000.0, 'interestIncome': None, 'totalOtherIncomeExpenseNet': 6783000.0, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': 21630000.0, 'taxProvision': None, 'incomeTaxExpense': 1021000.0, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': 20609000.0, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': 45568000.0, 'dilutedWeightedAverageShares': 129.212, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}","{'totalRevenue': 56616000.0, 'costOfRevenue': 2585000.0, 'grossProfit': 54031000.0, 'totalOperatingExpenses': 15485000.0,

In [40]:
#Ahora crearemos una función que nos permita obtener la cuenta de resultados de un listado de tickers
def obtencion_cuenta_pyg(listado_de_tickers):
    """
    Función que obtiene la cuenta de resultados de un listado de tickers y devuelve un dataframe con los datos.
    """
    #creamos un dataframe vacío para almacenar los datos
    df_cuentas_pyg = pd.DataFrame()
    #recorremos la lista de tickers y obtenemos los datos de cada uno
    for ticker in listado_de_tickers:
        #llamamos a la API de Profit.com para obtener los datos de la cuenta de resultados
        URL = f"https://api.profit.com/data-api/fundamentals/stocks/income_statement/{ticker}?token=" + api_key
        response = requests.get(URL)
        if response.status_code == 200:
            data = response.json()
            #convertimos el json a un dataframe y lo añadimos al dataframe final
            df_temp = pd.DataFrame(data).reset_index()
            #renombramos la columna index a 'fecha' y la convertimos en formato date time y eliminamos la columna 'index'
            df_temp = df_temp.rename(columns={"index":"fecha"})
            df_temp["fecha"] = pd.to_datetime(df_temp["fecha"])
            df_temp["ticker"] = ticker
            df_cuentas_pyg = pd.concat([df_cuentas_pyg, df_temp], ignore_index=True)
        else:
            print(f"Error: {response.status_code} - {response.text}")

    #devolvemos el dataframe con los datos de la cuenta de resultados y lo guardamos en un csv en la carpeta data_calculo_ratios
    return df_cuentas_pyg.to_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35.csv',index=False)

In [41]:
obtencion_cuenta_pyg(tickers)

In [62]:
#abrimos el csv que hemos guardado para ver los datos
df_cuentas_pyg = pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35.csv')
df_cuentas_pyg

,fecha,currency,quarterly,yearly,ticker
0,2000-06-30,EUR,"{'totalRevenue': None, 'costOfRevenue': None, 'grossProfit': None, 'totalOperatingExpenses': None, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': None, 'operatingIncome': None, 'interestIncome': None, 'totalOtherIncomeExpenseNet': None, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': None, 'taxProvision': None, 'incomeTaxExpense': None, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': None, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': None, 'dilutedWeightedAverageShares': None, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}",NaN,ACS.MC
1,2001-06-30,EUR,"{'totalRevenue': None, 'costOfRevenue': None, 'grossProfit': None, 'totalOperatingExpenses': None, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': None, 'operatingIncome': None, 'interestIncome': None, 'totalOtherIncomeExpenseNet': None, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': None, 'taxProvision': None, 'incomeTaxExpense': None, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': None, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': None, 'dilutedWeightedAverageShares': None, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}",NaN,ACS.MC
2,2001-12-31,EUR,"{'totalRevenue': None, 'costOfRevenue': None, 'grossProfit': None, 'totalOperatingExpenses': None, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': None, 'operatingIncome': None, 'interestIncome': None, 'totalOtherIncomeExpenseNet': None, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': None, 'taxProvision': None, 'incomeTaxExpense': None, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': None, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': None, 'dilutedWeightedAverageShares': None, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}","{'totalRevenue': 3921382000.0, 'costOfRevenue': None, 'grossProfit': None, 'totalOperatingExpenses': 267799000.0, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': 267799000.0, 'operatingIncome': 267799000.0, 'interestIncome': None, 'totalOtherIncomeExpenseNet': -45521000.0, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': 222278000.0, 'taxProvision': None, 'incomeTaxExpense': 70955000.0, 'extraordinaryItems': None, 'minorityInterest': 2092000.0, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': 149231000.0, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': 267799000.0, 'dilutedWeightedAverageShares': None, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}",ACS.MC
3,2002-06-30,EUR,"{'totalRevenue': None, 'costOfRevenue': None, 'grossProfit': None, 'totalOperatingExpenses': None, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': None, 'operatingIncome': None, 'intere

In [43]:
df_cuentas_pyg.describe(include='all').T

,count,unique,top,freq
fecha,2929,249,2019-12-31,34
currency,2920,2,EUR,2811
quarterly,2862,2650,"{'totalRevenue': None, 'costOfRevenue': None, 'grossProfit': None, 'totalOperatingExpenses': None, 'researchDevelopment': None, 'sellingGeneralAdministrative': None, 'discontinuedOperations': None, 'otherOperatingExpenses': None, 'operatingIncome': None, 'interestIncome': None, 'totalOtherIncomeExpenseNet': None, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': None, 'taxProvision': None, 'incomeTaxExpense': None, 'extraordinaryItems': None, 'minorityInterest': None, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': None, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': None, 'dilutedWeightedAverageShares': None, 'netIncomeAfterTaxes': None, 'netIncomeBeforeExtraordinaryItems': None, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0, 'dilutedNormalizedEPS': 0}",145
yearly,819,819,"{'totalRevenue': 2128430000.0, 'costOfRevenue': -18970000.0, 'grossProfit': 2147400000.0, 'totalOperatingExpenses': 1284729000.0, 'researchDevelopment': None, 'sellingGeneralAdministrative': 268000000.0, 'discontinuedOperations': None, 'otherOperatingExpenses': 1284729000.0, 'operatingIncome': 843701000.0, 'interestIncome': 2693000000.0, 'totalOtherIncomeExpenseNet': -27701000.0, 'nonOperatingIncomeNetOther': None, 'incomeBeforeTax': 816000000.0, 'taxProvision': 243000000.0, 'incomeTaxExpense': 243000000.0, 'extraordinaryItems': None, 'minorityInterest': 2000.0, 'effectOfAccountingCharges': None, 'nonRecurring': None, 'otherItems': None, 'netIncome': 573334000.0, 'preferredStockAndOtherAdjustments': None, 'netIncomeApplicableToCommonShares': None, 'ebit': -86850000.0, 'dilutedWeightedAverageShares': 2569.1575, 'netIncomeAfterTaxes': 573000000.0, 'netIncomeBeforeExtraordinaryItems': 573000000.0, 'dilutedNetIncome': None, 'dilutedEPSExcludingExtraordinaryItems': 0.22303031246624624, 'dilutedNormalizedEPS': 0}",1
ticker,2929,35,REP.MC,152


In [ ]:
#Ahora que tenemos los datos de la cuenta de resultados de todas las acciones del IBEX35, vamos a crear una nueva función que nos limpie el dataframe y nos deje solo los datos que nos interesan para calcular el PER.
#esto no lo hacemos directamente en la función de obtencion_cuenta_pyg para tener toda la cuenta y evitar llamar a la api más veces de las necesarias.
def limpieza_cuenta_pyg(dataframe_pyg=pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35.csv'), fecha_inicial = "2020-01-01"):
    #solo queremos los datos anuales desde la fecha inicial por lo que borramos los datos trimestrales y filtramos por la columna fecha
    dataframe_pyg = dataframe_pyg[dataframe_pyg["fecha"] >= fecha_inicial]
    dataframe_pyg.drop(columns=["quarterly"], inplace=True)
    #eliminamos las filas que no tienen datos en la columna 'yearly' ya que no nos interesan
    dataframe_pyg.dropna(subset=["yearly"], inplace=True)
    #Convertimos la columna fecha a datetime y sacamos el año de la fecha
    dataframe_pyg["fecha"] = pd.to_datetime(dataframe_pyg["fecha"])
    dataframe_pyg["Año"] = dataframe_pyg["fecha"].dt.year
    #eliminamos la columna fecha ya que no nos interesa
    dataframe_pyg.drop(columns=["fecha"], inplace=True)
    #la columna yearly contiene un diccionario con los datos de la cuenta de resultados. De estos datos nos interesan los siguientes:
    #'totalRevenue', 'netIncome'
    #convertimos la columna yearly a un diccionario para poder trabajar con ella
    dataframe_pyg["yearly"] = dataframe_pyg["yearly"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    #sacamos los datos de la columna yearly y los añadimos al dataframe como columnas nuevas
    dataframe_pyg["totalRevenue"] = dataframe_pyg["yearly"].apply(lambda x: x.get("totalRevenue"))
    dataframe_pyg["netIncome"] = dataframe_pyg["yearly"].apply(lambda x: x.get("netIncome"))
    #eliminamos la columna yearly y currency ya que no nos interesa
    dataframe_pyg.drop(columns=["yearly","currency"], inplace=True)

#la función devuelve el dataframe limpio y lo guardamos en un csv en la carpeta data_calculo_ratios
    return dataframe_pyg.to_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv',index=False)
    

In [84]:
limpieza_cuenta_pyg(df_cuentas_pyg)

C:\Users\ganad\AppData\Local\Temp\ipykernel_9312\1932145539.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_pyg.drop(columns=["quarterly"], inplace=True)
C:\Users\ganad\AppData\Local\Temp\ipykernel_9312\1932145539.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_pyg.dropna(subset=["yearly"], inplace=True)
C:\Users\ganad\AppData\Local\Temp\ipykernel_9312\1932145539.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [85]:
df_cuentas_pyg_limpio = pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv')
df_cuentas_pyg_limpio.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ticker,176,35,ITX.MC,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
netIncome,176.0,NaN,NaN,NaN,1353054886.363636,2920567723.518995,-7708000000.0,137771250.0,462604500.0,1404250000.0,20485000000.0
Año,176.0,NaN,NaN,NaN,2022.017045,1.432179,2020.0,2021.0,2022.0,2023.0,2025.0
totalRevenue,176.0,NaN,NaN,NaN,15642536579.545454,18651370449.146687,53266000.0,2178962000.0,6102833500.0,24824750000.0,79844000000.0


In [93]:
df_cuentas_pyg_limpio.groupby(["ticker"]).count().reset_index().sort_values("Año",ascending=False)


,ticker,netIncome,Año,totalRevenue
19,ITX.MC,6,6,6
1,ACX.MC,5,5,5
0,ACS.MC,5,5,5
2,AENA.MC,5,5,5
3,AMS.MC,5,5,5
5,ANE.MC,5,5,5
4,ANA.MC,5,5,5
7,BKT.MC,5,5,5
8,CABK.MC,5,5,5
9,CLNX.MC,5,5,5


In [95]:
#vemos que inditex tiene 6 años de datos y el resto 5, vamos a ver que sucede con los datos de inditex
df_itx = df_cuentas_pyg_limpio[df_cuentas_pyg_limpio["ticker"] == "ITX.MC"].sort_values("Año",ascending=False)
df_itx

,ticker,netIncome,Año,totalRevenue
100,ITX.MC,5.866133e+09,2025,3.863188e+10
99,ITX.MC,5.381000e+09,2024,3.594700e+10
98,ITX.MC,4.130000e+09,2023,3.256900e+10
97,ITX.MC,3.243000e+09,2022,2.771600e+10
96,ITX.MC,1.106000e+09,2021,2.040200e+10
95,ITX.MC,3.639000e+09,2020,2.828500e+10


Inditex tiene más datos que el resto porque su ejercicio no coincide con el año natural (acaba el 31/01/xxx). Para equipararlo con el resto de datos y poder calcular los ratios de forma correcta, eliminaremos los datos del año 2020 de inditex ya que son del 01/02/2019 al 31/01/2020 y haremos como si fueran año natural. Corregimos la función de limpieza para que lo haga directamente.

In [58]:
def limpieza_cuenta_pyg_def(dataframe_pyg=pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35.csv'), fecha_inicial = "2020-01-01"):
    """
    Función que limpia el dataframe de la cuenta de resultados y deja solo los datos que nos interesan para calcular el PER.
    """
    #solo queremos los datos anuales desde la fecha inicial por lo que borramos los datos trimestrales y filtramos por la columna fecha
    dataframe_pyg = dataframe_pyg[dataframe_pyg["fecha"] >= fecha_inicial]
    dataframe_pyg.drop(columns=["quarterly"], inplace=True)
    #eliminamos las filas que no tienen datos en la columna 'yearly' ya que no nos interesan
    dataframe_pyg.dropna(subset=["yearly"], inplace=True)
    #Convertimos la columna fecha a datetime y sacamos el año de la fecha
    dataframe_pyg["fecha"] = pd.to_datetime(dataframe_pyg["fecha"])
    dataframe_pyg["Año"] = dataframe_pyg["fecha"].dt.year
    #eliminamos la columna fecha ya que no nos interesa
    dataframe_pyg.drop(columns=["fecha"], inplace=True)
    #la columna yearly contiene un diccionario con los datos de la cuenta de resultados. De estos datos nos interesan los siguientes:
    #'totalRevenue', 'netIncome'
    #convertimos la columna yearly a un diccionario para poder trabajar con ella
    dataframe_pyg["yearly"] = dataframe_pyg["yearly"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    #sacamos los datos de la columna yearly y los añadimos al dataframe como columnas nuevas
    dataframe_pyg["totalRevenue"] = dataframe_pyg["yearly"].apply(lambda x: x.get("totalRevenue"))
    dataframe_pyg["netIncome"] = dataframe_pyg["yearly"].apply(lambda x: x.get("netIncome"))
    dataframe_pyg["EBIT"] = dataframe_pyg["yearly"].apply(lambda x: x.get("ebit"))
    #eliminamos la columna yearly y currency ya que no nos interesa
    dataframe_pyg.drop(columns=["yearly","currency"], inplace=True)
    #para los datos de inditex (ITX.MC) tenemos 6 años de datos, pero el resto de las acciones solo tienen 5 años. Por lo que vamos a eliminar los datos de inditex para el año 2020 y vamos a reducir el resto de valores de inditex 1 año, es decir los datos que ahora mismo tienen valor 2021 son de 2020 y así sucesivamente.
    dataframe_pyg = dataframe_pyg[~((dataframe_pyg["ticker"] == "ITX.MC") & (dataframe_pyg["Año"] == 2020))]
    dataframe_pyg.loc[dataframe_pyg["ticker"] == "ITX.MC", "Año"] -= 1
    

#la función devuelve el dataframe limpio y lo guardamos en un csv en la carpeta data_calculo_ratios
    return dataframe_pyg.to_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv',index=False)

In [59]:
limpieza_cuenta_pyg_def()
#abrimos el csv que hemos guardado para ver los datos
df_cuentas_pyg_limpio = pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv')
df_cuentas_pyg_limpio

C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\571847250.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_pyg.drop(columns=["quarterly"], inplace=True)
C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\571847250.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_pyg.dropna(subset=["yearly"], inplace=True)
C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\571847250.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

,ticker,Año,totalRevenue,netIncome,EBIT
0,ACS.MC,2020,3.493736e+10,8.960500e+08,1.085166e+09
1,ACS.MC,2021,2.783666e+10,-7.318700e+08,1.196940e+08
2,ACS.MC,2022,3.361523e+10,6.682270e+08,1.080231e+09
3,ACS.MC,2023,3.573776e+10,7.801230e+08,1.349721e+09
4,ACS.MC,2024,4.163312e+10,8.275800e+08,1.836656e+09
...,...,...,...,...,...
170,UNI.MC,2020,1.033204e+09,7.783100e+07,7.044500e+07
171,UNI.MC,2021,1.247682e+09,1.113202e+09,9.930610e+08
172,UNI.MC,2022,1.754926e+09,2.775760e+08,3.198630e+08
173,UNI.MC,2023,1.977103e+09,2.667030e+08,2.786230e+08


In [110]:
df_cuentas_pyg_limpio[df_cuentas_pyg_limpio["ticker"] == "ITX.MC"].sort_values("Año",ascending=False)

,ticker,Año,totalRevenue,netIncome
99,ITX.MC,2024,3.863188e+10,5.866133e+09
98,ITX.MC,2023,3.594700e+10,5.381000e+09
97,ITX.MC,2022,3.256900e+10,4.130000e+09
96,ITX.MC,2021,2.771600e+10,3.243000e+09
95,ITX.MC,2020,2.040200e+10,1.106000e+09


ya tenemos el beneficio por empresa. Para sacar el Per necesitamos el precio, el beneficio y el número de acciones en circulación, nos falta este último dato.

In [ ]:
#Sacamos usando la API de Profit.com las acciones en circulación de una empresa como ejemplo. estos datos los usaremos para calcular el PER y otras ratios
#En este caso usaremos la empresa 'MRL.MC' como ejemplo.
ticker = 'MRL.MC'
URL = f"https://api.profit.com/data-api/fundamentals/stocks/outstanding_shares/{ticker}?token=" + api_key
URL

'https://api.profit.com/data-api/fundamentals/stocks/outstanding_shares/MRL.MC?token=978336172b8c4b29960d9366896f6ef7'

In [112]:
#llamamos a la API de Profit.com para obtener los datos de las acciones en circulación
response = requests.get(URL)
if response.status_code == 200:
    data_MRL = response.json()
else:
    print(f"Error: {response.status_code} - {response.text}")
data_MRL

[{'year': 2024,
  'annual': 509.857,
  'q1': 462.7643,
  'q2': 468.4496,
  'q3': 562.5577,
  'q4': 509.857},
 {'year': 2023,
  'annual': 620.696,
  'q1': 472.6643,
  'q2': 463.9171,
  'q3': 316.0186,
  'q4': 620.696},
 {'year': 2022,
  'annual': 465.6524,
  'q1': 462.945,
  'q2': 462.945,
  'q3': 462.945,
  'q4': 465.6524},
 {'year': 2021,
  'annual': -4.6238,
  'q1': 464.3667,
  'q2': 1.5412,
  'q3': 481.2454,
  'q4': -4.6238},
 {'year': 2020,
  'annual': 464.9342,
  'q1': 482.875,
  'q2': 463.045,
  'q3': 467.6897,
  'q4': 464.9342},
 {'year': 2019,
  'annual': 462.8728,
  'q1': 462.2846,
  'q2': 473.4475,
  'q3': 464.6933,
  'q4': 462.8728},
 {'year': 2018,
  'annual': 475.5809,
  'q1': 475.6208,
  'q2': 461.1416,
  'q3': 468.6261,
  'q4': 475.5809},
 {'year': 2017,
  'annual': 459.6777,
  'q1': 475.3643,
  'q2': 462.7347,
  'q3': 466.4605,
  'q4': 459.6777},
 {'year': 2016,
  'annual': 394.4462,
  'q1': 323.1214,
  'q2': 322.9386,
  'q3': 321.983,
  'q4': 394.4462},
 {'year': 2015,

In [115]:
#lo transponemos a un dataframe para poder trabajar con él
df_MRL = pd.DataFrame(data_MRL)
df_MRL

,year,annual,q1,q2,q3,q4
0,2024,509.8570,462.7643,468.4496,562.5577,509.8570
1,2023,620.6960,472.6643,463.9171,316.0186,620.6960
2,2022,465.6524,462.9450,462.9450,462.9450,465.6524
3,2021,-4.6238,464.3667,1.5412,481.2454,-4.6238
4,2020,464.9342,482.8750,463.0450,467.6897,464.9342
5,2019,462.8728,462.2846,473.4475,464.6933,462.8728
6,2018,475.5809,475.6208,461.1416,468.6261,475.5809
7,2017,459.6777,475.3643,462.7347,466.4605,459.6777
8,2016,394.4462,323.1214,322.9386,321.9830,394.4462
9,2015,323.1214,130.6600,166.3134,680.4341,323.1214


In [62]:
#Creamos la función que nos permita obtener el número de acciones en circulación de un listado de tickers
def obtencion_acciones_circulacion(listado_de_tickers=tickers):
    #creamos un dataframe vacío para almacenar los datos
    df_acciones = pd.DataFrame()
    #recorremos la lista de tickers y obtenemos los datos de cada uno
    for ticker in listado_de_tickers:
        #llamamos a la API de Profit.com para obtener los datos del número de acciones en circulación
        URL = f"https://api.profit.com/data-api/fundamentals/stocks/outstanding_shares/{ticker}?token=" + api_key
        response = requests.get(URL)
        if response.status_code == 200:
            data = response.json()
            #convertimos el json a un dataframe y lo añadimos al dataframe final
            df_temp = pd.DataFrame(data)
            #eliminamos las columnas q1, q2, q3 y q4 ya que no nos interesan
            df_temp.drop(columns=["q1","q2","q3","q4"], inplace=True)
            #añadimos la columna ticker al dataframe
            df_temp["ticker"] = ticker
            df_acciones = pd.concat([df_acciones, df_temp], ignore_index=True)
        else:
            print(f"Error: {response.status_code} - {response.text}")
        #Solo queremos los datos de 2020 en adelante, por lo que filtramos el dataframe por la columna year
        df_acciones = df_acciones[df_acciones["year"] >= 2020]

        #Según la documentación de la API el total de acciones en circulación está en millones por lo que multiplicamos por 1.000.000 y creamos una nueva columna con el nombre 'Acciones Circulacion' y la añadimos al dataframe
        df_acciones["Acciones Circulacion"] = df_acciones["annual"] * 1000000


    #devolvemos el dataframe con los datos del número de acciones en circulación y lo guardamos en un csv en la carpeta data_calculo_ratios
    return df_acciones.to_csv('data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv',index=False)

In [63]:
obtencion_acciones_circulacion()

PermissionError: [Errno 13] Permission denied: 'data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv'

In [119]:
df_acciones_circulacion = pd.read_csv('data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv')
df_acciones_circulacion

,year,annual,ticker,Acciones Circulacion
0,2024,257.8144,ACS.MC,2.578144e+08
1,2023,261.2817,ACS.MC,2.612817e+08
2,2022,271.5823,ACS.MC,2.715823e+08
3,2021,300.6317,ACS.MC,3.006317e+08
4,2020,300.6317,ACS.MC,3.006317e+08
...,...,...,...,...
168,2024,2569.1575,UNI.MC,2.569158e+09
169,2023,2651.9062,UNI.MC,2.651906e+09
170,2022,2654.5484,UNI.MC,2.654548e+09
171,2021,2650.5016,UNI.MC,2.650502e+09


In [120]:
df_acciones_circulacion.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
year,173.0,NaN,NaN,NaN,2022.046243,1.45,2020.0,2021.0,2022.0,2023.0,2025.0
annual,173.0,NaN,NaN,NaN,2182.957182,3545.570186,-4.6238,261.2817,683.3062,3064.4242,24355.617
ticker,173,35,BKT.MC,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acciones Circulacion,173.0,NaN,NaN,NaN,2182957181.50289,3545570186.168964,-4623800.0,261281700.0,683306200.0,3064424200.0,24355617000.0


In [121]:
df_acciones_circulacion.groupby(["ticker"]).count().reset_index().sort_values("year",ascending=False)

,ticker,year,annual,Acciones Circulacion
7,BKT.MC,6,6,6
19,ITX.MC,6,6,6
0,ACS.MC,5,5,5
2,AENA.MC,5,5,5
1,ACX.MC,5,5,5
5,ANE.MC,5,5,5
3,AMS.MC,5,5,5
6,BBVA.MC,5,5,5
8,CABK.MC,5,5,5
9,CLNX.MC,5,5,5


In [125]:
df_acciones_circulacion[df_acciones_circulacion["ticker"].isin(["ITX.MC","BKT.MC"])]

,year,annual,ticker,Acciones Circulacion
35,2025,931.5000,BKT.MC,9.315000e+08
36,2024,898.6455,BKT.MC,8.986455e+08
37,2023,898.6285,BKT.MC,8.986285e+08
38,2022,898.6662,BKT.MC,8.986662e+08
39,2021,898.6385,BKT.MC,8.986385e+08
40,2020,897.9739,BKT.MC,8.979739e+08
96,2025,3120.2835,ITX.MC,3.120284e+09
97,2024,3113.5539,ITX.MC,3.113554e+09
98,2023,3113.0696,ITX.MC,3.113070e+09
99,2022,3111.7195,ITX.MC,3.111720e+09


Vemos que Bankinter e inditex nos devuelven valores para 2025. Como lo vamos a cruzar con los resultados, no nos altera nuestro análisis por lo que no es necesario borrarlo.


In [126]:
#Existen algunos números de acciones negativos, vamos a ver que sucede con ellos
df_acciones_circulacion[df_acciones_circulacion["Acciones Circulacion"] < 0]

,year,annual,ticker,Acciones Circulacion
115,2021,-4.6238,MRL.MC,-4623800.0


In [127]:
df_acciones_circulacion[df_acciones_circulacion["ticker"] == "MRL.MC"]

,year,annual,ticker,Acciones Circulacion
112,2024,509.8570,MRL.MC,509857000.0
113,2023,620.6960,MRL.MC,620696000.0
114,2022,465.6524,MRL.MC,465652400.0
115,2021,-4.6238,MRL.MC,-4623800.0
116,2020,464.9342,MRL.MC,464934200.0


investigando en internet, vemos que el número de acciones en circulación está bien pero no es negativo ya que no tiene ningun sentido financiero tener acciones en circulación negativas. Para mejorar el código y evitar esto aplico un absoluto al número de acciones en circulación. También cambiaré el token para evitar problemas de límite.

In [131]:
#Para sacar el número de acciones en circulación de la empresa "ITX.MC" vamos a usar la API de profit.com.
def obtencion_acciones_circulacion_def(listado_de_tickers=tickers):
    """
    Función que obtiene el número de acciones en circulación de un listado de tickers y devuelve un dataframe con los datos.
    """
    #creamos un dataframe vacío para almacenar los datos
    df_acciones = pd.DataFrame()
    #recorremos la lista de tickers y obtenemos los datos de cada uno
    for ticker in listado_de_tickers:
        #llamamos a la API de Profit.com para obtener los datos del número de acciones en circulación
        URL = f"https://api.profit.com/data-api/fundamentals/stocks/outstanding_shares/{ticker}?token=" + otro_token
        response = requests.get(URL)
        if response.status_code == 200:
            data = response.json()
            #convertimos el json a un dataframe y lo añadimos al dataframe final
            df_temp = pd.DataFrame(data)
            #eliminamos las columnas q1, q2, q3 y q4 ya que no nos interesan
            df_temp.drop(columns=["q1","q2","q3","q4"], inplace=True)
            #añadimos la columna ticker al dataframe
            df_temp["ticker"] = ticker
            df_acciones = pd.concat([df_acciones, df_temp], ignore_index=True)
        else:
            print(f"Error: {response.status_code} - {response.text}")
        #Solo queremos los datos de 2020 en adelante, por lo que filtramos el dataframe por la columna year
        df_acciones = df_acciones[df_acciones["year"] >= 2020]

        #Aplicamos el valor absoluto a la columna annual para evitar problemas con los números negativos
        df_acciones["annual"] = df_acciones["annual"].abs()

        #Según la documentación de la API el total de acciones en circulación está en millones por lo que multiplicamos por 1.000.000 y creamos una nueva columna con el nombre 'Acciones Circulacion' y la añadimos al dataframe
        df_acciones["Acciones Circulacion"] = df_acciones["annual"] * 1000000


    #devolvemos el dataframe con los datos del número de acciones en circulación y lo guardamos en un csv en la carpeta data_calculo_ratios
    return df_acciones.to_csv('data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv',index=False)

In [ ]:
obtencion_acciones_circulacion_def()


,year,annual,ticker,Acciones Circulacion
0,2024,257.8144,ACS.MC,2.578144e+08
1,2023,261.2817,ACS.MC,2.612817e+08
2,2022,271.5823,ACS.MC,2.715823e+08
3,2021,300.6317,ACS.MC,3.006317e+08
4,2020,300.6317,ACS.MC,3.006317e+08
...,...,...,...,...
168,2024,2569.1575,UNI.MC,2.569158e+09
169,2023,2651.9062,UNI.MC,2.651906e+09
170,2022,2654.5484,UNI.MC,2.654548e+09
171,2021,2650.5016,UNI.MC,2.650502e+09


In [3]:
#abrimos el csv que hemos guardado para ver los datos
df_acciones_circulacion = pd.read_csv('data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv')
df_acciones_circulacion

,year,annual,ticker,Acciones Circulacion
0,2024,257.8144,ACS.MC,2.578144e+08
1,2023,261.2817,ACS.MC,2.612817e+08
2,2022,271.5823,ACS.MC,2.715823e+08
3,2021,300.6317,ACS.MC,3.006317e+08
4,2020,300.6317,ACS.MC,3.006317e+08
...,...,...,...,...
168,2024,2569.1575,UNI.MC,2.569158e+09
169,2023,2651.9062,UNI.MC,2.651906e+09
170,2022,2654.5484,UNI.MC,2.654548e+09
171,2021,2650.5016,UNI.MC,2.650502e+09


In [133]:
df_acciones_circulacion.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
year,173.0,NaN,NaN,NaN,2022.046243,1.45,2020.0,2021.0,2022.0,2023.0,2025.0
annual,173.0,NaN,NaN,NaN,2183.010636,3545.537083,4.6238,261.2817,683.3062,3064.4242,24355.617
ticker,173,35,BKT.MC,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acciones Circulacion,173.0,NaN,NaN,NaN,2183010635.838151,3545537083.191043,4623800.0,261281700.0,683306200.0,3064424200.0,24355617000.0


In [ ]:
dataframe_pyg=pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv'), ,

In [149]:
dataframe_acciones=pd.read_csv('data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv')
dataframe_acciones

,year,annual,ticker,Acciones Circulacion
0,2024,257.8144,ACS.MC,2.578144e+08
1,2023,261.2817,ACS.MC,2.612817e+08
2,2022,271.5823,ACS.MC,2.715823e+08
3,2021,300.6317,ACS.MC,3.006317e+08
4,2020,300.6317,ACS.MC,3.006317e+08
...,...,...,...,...
168,2024,2569.1575,UNI.MC,2.569158e+09
169,2023,2651.9062,UNI.MC,2.651906e+09
170,2022,2654.5484,UNI.MC,2.654548e+09
171,2021,2650.5016,UNI.MC,2.650502e+09


In [172]:
#Con todo esto ya tenemos los datos necesarios para calcular el PER. Ahora vamos a crear una función que nos calcule el PER de cada acción y lo añada a un data frame nuevo donde guardaremos todos los ratios que calculemos.
def calculo_PER_ratio(dataframe_pyg=pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv'), dataframe_acciones=pd.read_csv('data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv'),dataframe_precios=pd.read_csv('data_calculo_ratios/01.PER.cotizaciones_IBEX35_ultimos_precios.csv')):
    """
    Función que calcula el PER de cada acción y lo añade a un dataframe nuevo donde guardaremos todos los ratios que calculemos.
    """
    #a la columna "year" del dataframe de acciones en circulación le cambiamos el nombre a "Año" para poder hacer el merge con el dataframe de la cuenta de resultados
    dataframe_acciones.rename(columns={"year":"Año"}, inplace=True)
    #creamos dos dataframe vacío para almacenar los datos
    df_ratios = pd.DataFrame()
    df_ratios_unif = pd.DataFrame()
    #a este dataframe le añadimos los datos de la cuenta de resultados, los precios y el número de acciones en circulación por ticker y año
    df_ratios_unif = pd.merge(dataframe_pyg, dataframe_acciones, on=["ticker","Año"], how="inner")
    df_ratios_unif = pd.merge(df_ratios_unif, dataframe_precios, on=["ticker","Año"], how="inner")
    #calculamos el BPA y el PER y los añadimos al dataframe
    df_ratios_unif["BPA"] = df_ratios_unif["netIncome"] / df_ratios_unif["Acciones Circulacion"]
    df_ratios_unif["PER"] = df_ratios_unif["Ultimo Precio"] / df_ratios_unif["BPA"]
    #unficamos este dataframe al vacio para limpiar el otro data frame y guardar ambos en distintos csv
    df_ratios = df_ratios_unif[["ticker","Año","PER"]]



    return df_ratios.to_csv('data_calculo_ratios/00.ratios_totales_IBEX35.csv',index=False), df_ratios_unif.to_csv('data_calculo_ratios/01.PER_IBEX35.csv',index=False)



In [ ]:
calculo_PER_ratio()



,ticker,Año,PER
0,ACS.MC,2020,6.763836
1,ACS.MC,2021,-7.718406
2,ACS.MC,2022,9.477766
3,ACS.MC,2023,12.512750
4,ACS.MC,2024,14.950232
...,...,...,...
165,UNI.MC,2020,11.568167
166,UNI.MC,2021,1.666680
167,UNI.MC,2022,8.224456
168,UNI.MC,2023,7.755769


In [9]:
#abrimos el csv que hemos guardado para ver los datos
df_ratios = pd.read_csv('data_calculo_ratios/00.ratios_totales_IBEX35.csv')
df_ratios

,ticker,Año,PER
0,ACS.MC,2020,6.763836
1,ACS.MC,2021,-7.718406
2,ACS.MC,2022,9.477766
3,ACS.MC,2023,12.512750
4,ACS.MC,2024,14.950232
...,...,...,...
165,UNI.MC,2020,11.568167
166,UNI.MC,2021,1.666680
167,UNI.MC,2022,8.224456
168,UNI.MC,2023,7.755769


Ya tenemos el PER de todas las acciones. Vemos que hay acciones que en momentos puntuales tienen un PER negativo. Esto se debe a que han tenido pérdidas en ese año, no es un error de dato. Ya veremos más adelante si, en estos casos, tener un dato de un PER negativo es útil para nosotros. 

Continuamos con la extracción de ratios. Vamos con el ROE. Para la obtención del ROE (Beneficio neto/Patrimonio Neto) necesatimos obtener el beneficio neto que ya lo hemos sacado anteriormente y lo tenemos guardado en el fichero "01.PER.cuentas_pyg_IBEX35_limpio" Y el Patrimonio Neto que tiraremos de Profit.com para obtenerlo

In [10]:
#Vamos a crear la URL para obtener el balance de una empresa como ejemplo. estos datos los usaremos para calcular el ROE y otras ratios
#En primer lugar definiremos la url
ticker = 'MRL.MC'
URL = f"https://api.profit.com/data-api/fundamentals/stocks/balance_sheet/{ticker}?token=" + api_key
URL

'https://api.profit.com/data-api/fundamentals/stocks/balance_sheet/MRL.MC?token=978336172b8c4b29960d9366896f6ef7'

In [11]:
#Creamos la respuesta de la API y la guardamos en un dataframe para poder trabajar con ella
response = requests.get(URL)
if response.status_code == 200:
    data_MRL = response.json()
else: 
    print(f"Error: {response.status_code} - {response.text}")
data_MRL
#Convertimos el json a un dataframe para poder trabajar con él
df_MRL = pd.DataFrame(data_MRL).reset_index()
df_MRL

,index,currency,quarterly,yearly
0,2014-03-31,EUR,"{'totalAssets': 1716477000.0, 'totalCurrentAssets': 11135000.0, 'cashAndShortTermInvestments': 10955000.0, 'cash': 10858000.0, 'shortTermInvestments': 97000.0, 'netReceivables': None, 'inventory': None, 'otherCurrentAssets': 180000.0, 'nonCurrrentAssetsOther': 1705255000.0, 'propertyPlantEquipment': None, 'propertyPlantAndEquipmentGross': None, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': None, 'intangibleAssets': None, 'otherAssets': None, 'netTangibleAssets': None, 'nonCurrentAssetsTotal': 1705342000.0, 'liabilitiesAndStockholdersEquity': 1716477000.0, 'totalLiab': 964639000.0, 'totalCurrentLiabilities': 32631000.0, 'shortTermDebt': 30711000.0, 'shortLongTermDebtTotal': 853654000.0, 'capitalLeaseObligations': None, 'accountsPayable': 1920000.0, 'otherCurrentLiab': None, 'nonCurrentLiabilitiesTotal': 932008000.0, 'longTermDebtTotal': None, 'longTermDebt': None, 'otherLiab': None, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': None, 'totalStockholderEquity': 751838000.0, 'commonStockTotalEquity': None, 'commonStock': None, 'commonStockSharesOutstanding': 170291389.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': None, 'accumulatedOtherComprehensiveIncome': None, 'otherStockholderEquity': 751838000.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': None, 'date': '2014-03-31', 'filing_date': '2014-03-31', 'netDebt': 842796000.0, 'earningAssets': None, 'netWorkingCapital': None, 'netInvestedCapital': None}",NaN
1,2014-06-30,EUR,"{'totalAssets': 1250602000.0, 'totalCurrentAssets': 1250600000.0, 'cashAndShortTermInvestments': 1250519000.0, 'cash': 1250519000.0, 'shortTermInvestments': None, 'netReceivables': None, 'inventory': None, 'otherCurrentAssets': 81000.0, 'nonCurrrentAssetsOther': None, 'propertyPlantEquipment': None, 'propertyPlantAndEquipmentGross': None, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': None, 'intangibleAssets': None, 'otherAssets': None, 'netTangibleAssets': None, 'nonCurrentAssetsTotal': 2000.0, 'liabilitiesAndStockholdersEquity': 1250602000.0, 'totalLiab': 28998000.0, 'totalCurrentLiabilities': 28998000.0, 'shortTermDebt': None, 'shortLongTermDebtTotal': None, 'capitalLeaseObligations': None, 'accountsPayable': 28997000.0, 'otherCurrentLiab': 1000.0, 'nonCurrentLiabilitiesTotal': None, 'longTermDebtTotal': None, 'longTermDebt': None, 'otherLiab': 1000.0, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': None, 'totalStockholderEquity': 1221604000.0, 'commonStockTotalEquity': None, 'commonStock': 125060000.0, 'commonStockSharesOutstanding': 141920839.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': -35000.0, 'accumulatedOtherComprehensiveIncome': -28420999.0, 'otherStockholderEquity': 1124999999.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': None, 'date': '2014-06-30', 'filing_date': '2014-06-30', 'netDebt': -1250519000.0, 'earningAssets': None, 'netWorkingCapital': None, 'netInvestedCapital': None}",NaN
2,2014-09-30,EUR,"{'totalAssets': 2231707000.0, 'totalCurrentAssets': 208946000.0, 'cashAndShortTermInvestments': 206880000.0, 'cash': 79929000.0, 'shortTermInvestments': 126951000.0, 'netReceivables': None, 'inventory': None, 'otherCurrentAssets': 2066000.0, 'nonC

In [48]:
df_MRL.rename(columns={"index":"fecha"}, inplace=True)
df_MRL["fecha"] = pd.to_datetime(df_MRL["fecha"])
#eliminamos la columna quarterly ya que no nos interesa

#solo nos interesan los datos anuales por lo que eliminamos los trimestrales
df_MRL.dropna(subset=["yearly"], inplace=True)
#filtramos por la fecha para quedarnos solo con los datos de 2020 en adelante
df_MRL = df_MRL[df_MRL["fecha"]>="2020-01-01"]
#los datos que nos interesan están dentro de la columna yearly, por lo que convertimos esta columna a un diccionario para poder trabajar con ella
df_MRL["yearly"] = df_MRL["yearly"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
#sacamos los siguientes datos de la columna yearly y los añadimos al dataframe como columnas nuevas: totalStockholderEquity, totalLiab, accountsPayable, cash, totalAssets y las convertimos a tipo float
df_MRL["totalPatrimonioNeto"] = df_MRL["yearly"].apply(lambda x: x.get("totalStockholderEquity"))
df_MRL["totalPasivo"] = df_MRL["yearly"].apply(lambda x: x.get("totalLiab"))
df_MRL["cuentasAPagar_proveedores"] = df_MRL["yearly"].apply(lambda x: x.get("accountsPayable"))
df_MRL["efectivo"] = df_MRL["yearly"].apply(lambda x: x.get("cash"))
df_MRL["totalActivos"] = df_MRL["yearly"].apply(lambda x: x.get("totalAssets"))
#Sacamos el año de la fecha y lo añadimos al dataframe como una nueva columna
df_MRL["año"] = df_MRL["fecha"].dt.year
df_MRL

,fecha,currency,yearly,totalPatrimonioNeto,totalPasivo,cuentasAPagar_proveedores,efectivo,totalActivos,año
27,2020-12-31,EUR,"{'totalAssets': 13477612000.0, 'totalCurrentAssets': 415855000.0, 'cashAndShortTermInvestments': 259620000.0, 'cash': 252022000.0, 'shortTermInvestments': 7598000.0, 'netReceivables': 105135000.0, 'inventory': 33436000.0, 'otherCurrentAssets': 17664000.0, 'nonCurrrentAssetsOther': 12433322000.0, 'propertyPlantEquipment': 12146453000.0, 'propertyPlantAndEquipmentGross': None, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': None, 'intangibleAssets': 961000.0, 'otherAssets': 373846000.0, 'netTangibleAssets': 6695306000.0, 'nonCurrentAssetsTotal': 13061757000.0, 'liabilitiesAndStockholdersEquity': 13477612000.0, 'totalLiab': 6781345000.0, 'totalCurrentLiabilities': 179260000.0, 'shortTermDebt': 48275000.0, 'shortLongTermDebtTotal': 5806174000.0, 'capitalLeaseObligations': None, 'accountsPayable': 49794000.0, 'otherCurrentLiab': 15408000.0, 'nonCurrentLiabilitiesTotal': 6602085000.0, 'longTermDebtTotal': 5757899000.0, 'longTermDebt': 5719880000.0, 'otherLiab': 844186000.0, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': 46260000.0, 'totalStockholderEquity': 6696267000.0, 'commonStockTotalEquity': None, 'commonStock': 469771000.0, 'commonStockSharesOutstanding': 469993000.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': 56358000.0, 'accumulatedOtherComprehensiveIncome': 2467236000.0, 'otherStockholderEquity': 3702902000.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': 3813409000.0, 'date': '2020-12-31', 'filing_date': '2020-12-31', 'netDebt': 5432657000.0, 'earningAssets': None, 'netWorkingCapital': 236595000.0, 'netInvestedCapital': 12464422000.0}",6.696267e+09,6.781345e+09,49794000.0,2.520220e+08,1.347761e+10,2020
31,2021-12-31,EUR,"{'totalAssets': 14272837000.0, 'totalCurrentAssets': 1038443000.0, 'cashAndShortTermInvestments': 942868000.0, 'cash': 860187000.0, 'shortTermInvestments': 82681000.0, 'netReceivables': 46396000.0, 'inventory': 38697000.0, 'otherCurrentAssets': 10482000.0, 'nonCurrrentAssetsOther': 12732933000.0, 'propertyPlantEquipment': 12306417000.0, 'propertyPlantAndEquipmentGross': None, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': None, 'intangibleAssets': 1594000.0, 'otherAssets': 436803000.0, 'netTangibleAssets': 7025328000.0, 'nonCurrentAssetsTotal': 13234394000.0, 'liabilitiesAndStockholdersEquity': 14272837000.0, 'totalLiab': 7245915000.0, 'totalCurrentLiabilities': 736630000.0, 'shortTermDebt': 601719000.0, 'shortLongTermDebtTotal': 6260428000.0, 'capitalLeaseObligations': None, 'accountsPayable': 45761000.0, 'otherCurrentLiab': 14163000.0, 'nonCurrentLiabilitiesTotal': 6509285000.0, 'longTermDebtTotal': 5658709000.0, 'longTermDebt': 5584294000.0, 'otherLiab': 850576000.0, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': 46134000.0, 'totalStockholderEquity': 7026922000.0, 'commonStockTotalEquity': None, 'commonStock': 469771000.0, 'commonStockSharesOutstanding': 466396524.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': 512217000.0, 'accumulatedOtherComprehensiveIncome': 2941580000.0, 'otherStockholderEquity': 3103354000.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': 3647876000.0, 'date': '2

In [ ]:
#Accedemos a los datos de la columna yearly del "2024-12-31" y los convertimos a un dataframe para poder ver los datos que tiene y que es lo que nos interesa
data_prueba = df_MRL.iloc[43,3]
data_prueba


{'totalAssets': 13459195000.0,
 'totalCurrentAssets': 1700790000.0,
 'cashAndShortTermInvestments': 1561879000.0,
 'cash': 1550456000.0,
 'shortTermInvestments': 11423000.0,
 'netReceivables': 60338000.0,
 'inventory': 54005000.0,
 'otherCurrentAssets': 24568000.0,
 'nonCurrrentAssetsOther': 11095686000.0,
 'propertyPlantEquipment': None,
 'propertyPlantAndEquipmentGross': None,
 'accumulatedDepreciation': None,
 'longTermInvestments': None,
 'goodWill': None,
 'intangibleAssets': 1025000.0,
 'otherAssets': None,
 'netTangibleAssets': None,
 'nonCurrentAssetsTotal': 11758405000.0,
 'liabilitiesAndStockholdersEquity': 13459195000.0,
 'totalLiab': 5958215000.0,
 'totalCurrentLiabilities': 840253000.0,
 'shortTermDebt': 625766000.0,
 'shortLongTermDebtTotal': 4922456000.0,
 'capitalLeaseObligations': None,
 'accountsPayable': 117032000.0,
 'otherCurrentLiab': 90596000.0,
 'nonCurrentLiabilitiesTotal': 5117962000.0,
 'longTermDebtTotal': None,
 'longTermDebt': 4296690000.0,
 'otherLiab': N

In [26]:
df_prueba = pd.DataFrame([data_prueba])
df_prueba

,totalAssets,totalCurrentAssets,cashAndShortTermInvestments,cash,shortTermInvestments,netReceivables,inventory,otherCurrentAssets,nonCurrrentAssetsOther,propertyPlantEquipment,propertyPlantAndEquipmentGross,accumulatedDepreciation,longTermInvestments,goodWill,intangibleAssets,otherAssets,netTangibleAssets,nonCurrentAssetsTotal,liabilitiesAndStockholdersEquity,totalLiab,totalCurrentLiabilities,shortTermDebt,shortLongTermDebtTotal,capitalLeaseObligations,accountsPayable,otherCurrentLiab,nonCurrentLiabilitiesTotal,longTermDebtTotal,longTermDebt,otherLiab,deferredLongTermLiab,deferredLongTermAssetCharges,nonCurrentLiabilitiesOther,totalStockholderEquity,commonStockTotalEquity,commonStock,commonStockSharesOutstanding,additionalPaidInCapital,treasuryStock,preferredStockTotalEquity,preferredStockRedeemable,retainedEarningsTotalEquity,retainedEarnings,accumulatedOtherComprehensiveIncome,otherStockholderEquity,totalPermanentEquity,noncontrollingInterestInConsolidatedEntity,temporaryEquityRedeemableNoncontrollingInterests,accumulatedAmortization,negativeGoodwill,warrants,capitalSurpluse,date,filing_date,netDebt,earningAssets,netWorkingCapital,netInvestedCapital
0,1.345920e+10,1.700790e+09,1.561879e+09,1.550456e+09,11423000.0,60338000.0,54005000.0,24568000.0,1.109569e+10,None,None,None,None,None,1025000.0,None,None,1.175840e+10,1.345920e+10,5.958215e+09,840253000.0,625766000.0,4.922456e+09,None,117032000.0,90596000.0,5.117962e+09,None,4.296690e+09,None,None,None,73820000.0,7.500980e+09,None,563725000.0,509857000.0,None,None,None,None,None,283759000.0,None,6.653496e+09,None,None,None,None,None,None,None,2024-12-31,2024-12-31,3.372000e+09,None,860537000.0,1.242344e+10


Las columnas que nos interesan son las siguientes: 

    Patrimonio Neto - totalStockholderEquity
    Total pasivo - totalLiab
    Proveedores - accountsPayable
    Caja- cash
    Total activo - totalAssets

Vamos a añadir un pequeño control contable revisando que el total patrimonio neto + total pasivo sea igual al total activo


In [21]:
#Ahra creamos una función que nos permita obtener el balance de un listado de tickers
def obtencion_balance(listado_de_tickers):
    """
    Función que obtiene el balance de un listado de tickers y devuelve un dataframe con los datos.
    """
    #creamos un dataframe vacío para almacenar los datos
    df_balance = pd.DataFrame()
    #recorremos la lista de tickers y obtenemos los datos de cada uno
    for ticker in listado_de_tickers:
        #llamamos a la API de Profit.com para obtener los datos del balance
        URL = f"https://api.profit.com/data-api/fundamentals/stocks/balance_sheet/{ticker}?token=" + otro_token
        response = requests.get(URL)
        if response.status_code == 200:
            data = response.json()
            #convertimos el json a un dataframe y lo añadimos al dataframe final
            df_temp = pd.DataFrame(data).reset_index()
            #añadimos la columna ticker al dataframe
            df_temp["ticker"] = ticker
            df_balance = pd.concat([df_balance, df_temp], ignore_index=True)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break

    
    return df_balance.to_csv('data_calculo_ratios/02.ROE.balance_IBEX35.csv',index=False)

In [46]:
#Creamos una funcion que limpie el dataframe y nos deje solo los datos que nos interesan para calcular el ROE.
def limpieza_balance(df_balance=pd.read_csv('data_calculo_ratios/02.ROE.balance_IBEX35.csv'), fecha_inicial = "2020-01-01"):
    """
    Función que limpia el dataframe del balance y deja solo los datos que nos interesan para calcular el ROE.
    """
    #renombramos la columna index a 'fecha' y la convertimos en formato date time y eliminamos la columna 'index'
    df_balance.rename(columns={"index":"fecha"}, inplace=True)
    #convertimos la columna fecha a datetime
    df_balance["fecha"] = pd.to_datetime(df_balance["fecha"])
    #eliminamos la columna quarterly ya que no nos interesa
    df_balance.drop(columns=["quarterly"], inplace=True)
    #solo nos interesan los datos anuales por lo que eliminamos los trimestrales
    df_balance.dropna(subset=["yearly"], inplace=True)
    #filtramos por la fecha para quedarnos solo con los datos de 2020 en adelante
    df_balance = df_balance[df_balance["fecha"]>=fecha_inicial]
    #los datos que nos interesan están dentro de la columna yearly, por lo que convertimos esta columna a un diccionario para poder trabajar con ella
    df_balance["yearly"] = df_balance["yearly"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    #sacamos los siguientes datos de la columna yearly y los añadimos al dataframe como columnas nuevas: totalStockholderEquity, totalLiab, accountsPayable, cash, totalAssets y las convertimos a tipo float
    df_balance["totalPatrimonioNeto+pasivo"] = df_balance["yearly"].apply(lambda x: x.get("liabilitiesAndStockholdersEquity"))
    df_balance["totalPatrimonioNeto"] = df_balance["yearly"].apply(lambda x: x.get("totalStockholderEquity"))
    df_balance["totalPasivo"] = df_balance["totalPatrimonioNeto+pasivo"] - df_balance["totalPatrimonioNeto"]    
    df_balance["cuentasAPagar_proveedores"] = df_balance["yearly"].apply(lambda x: x.get("accountsPayable"))
    df_balance["efectivo"] = df_balance["yearly"].apply(lambda x: x.get("cash"))
    df_balance["totalActivos"] = df_balance["yearly"].apply(lambda x: x.get("totalAssets"))
    #Sacamos el año de la fecha y lo añadimos al dataframe como una nueva columna
    df_balance["Año"] = df_balance["fecha"].dt.year
    #eliminamos la columna fecha ya que no nos interesa
    df_balance.drop(columns=["fecha"], inplace=True)
    #hacemos una pequeña comprobación contable, la columna totalActivos debe ser igual a la suma de totalPasivo y totalPatrimonioNeto
    df_balance["comprobacion"] = df_balance["totalActivos"] - df_balance["totalPatrimonioNeto+pasivo"]
    #si la comprobacion es un valor vacio significa que no hay datos por lo que la eliminamos
    df_balance.dropna(subset=["comprobacion"], inplace=True)
    #si la comprobacion es 0, es que los datos son correctos, si no es 0, es que hay un error en los datos
    df_balance["comprobacion_2"] = df_balance["comprobacion"].apply(lambda x: "Correcto" if x == 0 else "Error")
    #devolvemos el dataframe con los datos del balance y lo guardamos en un csv en la carpeta data_calculo_ratios

    return df_balance.to_csv('data_calculo_ratios/02.ROE.balance_IBEX35_limpio.csv',index=False)


In [19]:
#llamamos a la función para obtener el balance de los tickers del ibex35
obtencion_balance(tickers)

In [50]:
#abrimos el csv que hemos guardado para ver los datos
df_balance_prueba = pd.read_csv('data_calculo_ratios/02.ROE.balance_IBEX35.csv')
df_balance_prueba

,index,currency,quarterly,yearly,ticker
0,2000-06-30,EUR,"{'totalAssets': 3192480000.0, 'totalCurrentAssets': None, 'cashAndShortTermInvestments': None, 'cash': None, 'shortTermInvestments': None, 'netReceivables': 1957220000.0, 'inventory': 164410000.0, 'otherCurrentAssets': 10820000.0, 'nonCurrrentAssetsOther': None, 'propertyPlantEquipment': 439840000.0, 'propertyPlantAndEquipmentGross': None, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': None, 'intangibleAssets': 125460000.0, 'otherAssets': 413540000.0, 'netTangibleAssets': None, 'nonCurrentAssetsTotal': None, 'liabilitiesAndStockholdersEquity': None, 'totalLiab': 2416580000.0, 'totalCurrentLiabilities': 1957980000.0, 'shortTermDebt': None, 'shortLongTermDebtTotal': None, 'capitalLeaseObligations': None, 'accountsPayable': 1324670000.0, 'otherCurrentLiab': 301740000.0, 'nonCurrentLiabilitiesTotal': None, 'longTermDebtTotal': None, 'longTermDebt': 213410000.0, 'otherLiab': None, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': None, 'totalStockholderEquity': 756540000.0, 'commonStockTotalEquity': None, 'commonStock': None, 'commonStockSharesOutstanding': None, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': None, 'accumulatedOtherComprehensiveIncome': None, 'otherStockholderEquity': None, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': None, 'date': '2000-06-30', 'filing_date': None, 'netDebt': None, 'earningAssets': None, 'netWorkingCapital': None, 'netInvestedCapital': None}",NaN,ACS.MC
1,2001-06-30,EUR,"{'totalAssets': 3478796000.0, 'totalCurrentAssets': None, 'cashAndShortTermInvestments': None, 'cash': None, 'shortTermInvestments': None, 'netReceivables': 2001527000.0, 'inventory': 189211000.0, 'otherCurrentAssets': 13871000.0, 'nonCurrrentAssetsOther': None, 'propertyPlantEquipment': 522532000.0, 'propertyPlantAndEquipmentGross': None, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': None, 'intangibleAssets': 126957000.0, 'otherAssets': 436346000.0, 'netTangibleAssets': None, 'nonCurrentAssetsTotal': None, 'liabilitiesAndStockholdersEquity': None, 'totalLiab': 2598457000.0, 'totalCurrentLiabilities': 2173957000.0, 'shortTermDebt': None, 'shortLongTermDebtTotal': None, 'capitalLeaseObligations': None, 'accountsPayable': 1551801000.0, 'otherCurrentLiab': 388471000.0, 'nonCurrentLiabilitiesTotal': None, 'longTermDebtTotal': None, 'longTermDebt': 382039000.0, 'otherLiab': None, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': None, 'totalStockholderEquity': 855571000.0, 'commonStockTotalEquity': None, 'commonStock': 96093000.0, 'commonStockSharesOutstanding': None, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': None, 'accumulatedOtherComprehensiveIncome': None, 'otherStockholderEquity': None, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': None, 'date': '2001-06-30', 'filing_date': None, 'netDebt': None, 'earningAssets': None, 'netWorkingCapital': None, 'netInvestedCapital': None}",NaN,ACS.MC
2,2001-12-31,EUR,"{'totalAssets': 3880591000.0, 'totalCurrentAssets': None, 'cashAndShortTermInvestments': None, 'cash': None, 'shortTermInvestments': None, 'netReceivables': 2176848000.0, 'inventory': 184138000.0, 'otherCurrentAssets': 15933000.0, 'nonCurrrentAssetsOther': None, 'propertyPlantEquipment

In [51]:
#ejecutamos la función de limpieza del balance para dejarlo limpio y guardarlo en un csv
limpieza_balance(df_balance_prueba)
#abrimos el csv que hemos guardado para ver los datos
df_balance = pd.read_csv('data_calculo_ratios/02.ROE.balance_IBEX35_limpio.csv')
df_balance

C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\2015286705.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_balance["yearly"] = df_balance["yearly"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\2015286705.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_balance["totalPatrimonioNeto+pasivo"] = df_balance["yearly"].apply(lambda x: x.get("liabilitiesAndStockholdersEquity"))
C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\2

,currency,yearly,ticker,totalPatrimonioNeto+pasivo,totalPatrimonioNeto,totalPasivo,cuentasAPagar_proveedores,efectivo,totalActivos,Año,comprobacion,comprobacion_2
0,EUR,"{'totalAssets': 37333720000.0, 'totalCurrentAssets': 24314925000.0, 'cashAndShortTermInvestments': 8956035000.0, 'cash': 7765477000.0, 'shortTermInvestments': 1190558000.0, 'netReceivables': 14410495000.0, 'inventory': 715241000.0, 'otherCurrentAssets': 233154000.0, 'nonCurrrentAssetsOther': 1463741000.0, 'propertyPlantEquipment': 2007922000.0, 'propertyPlantAndEquipmentGross': 4848782000.0, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': 2863053000.0, 'intangibleAssets': 796126000.0, 'otherAssets': 2381225000.0, 'netTangibleAssets': -130700000.0, 'nonCurrentAssetsTotal': 13018795000.0, 'liabilitiesAndStockholdersEquity': 37333720000.0, 'totalLiab': 33057813000.0, 'totalCurrentLiabilities': 22451823000.0, 'shortTermDebt': 3003440000.0, 'shortLongTermDebtTotal': 11846312000.0, 'capitalLeaseObligations': 688087000.0, 'accountsPayable': 8619276000.0, 'otherCurrentLiab': 226888000.0, 'nonCurrentLiabilitiesTotal': 10605990000.0, 'longTermDebtTotal': 8842872000.0, 'longTermDebt': 8288481000.0, 'otherLiab': 1763118000.0, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': 112979000.0, 'totalStockholderEquity': 3528479000.0, 'commonStockTotalEquity': None, 'commonStock': 155332000.0, 'commonStockSharesOutstanding': 293847994.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': 574005000.0, 'accumulatedOtherComprehensiveIncome': 3513932000.0, 'otherStockholderEquity': -714790000.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': 495226000.0, 'date': '2020-12-31', 'filing_date': '2020-12-31', 'netDebt': 4017203000.0, 'earningAssets': None, 'netWorkingCapital': 1863102000.0, 'netInvestedCapital': 14623072000.0}",ACS.MC,3.733372e+10,3.528479e+09,3.380524e+10,8.619276e+09,7.765477e+09,3.733372e+10,2020,0.0,Correcto
1,EUR,"{'totalAssets': 35664345000.0, 'totalCurrentAssets': 24654314000.0, 'cashAndShortTermInvestments': 11978647000.0, 'cash': 10941027000.0, 'shortTermInvestments': 1037620000.0, 'netReceivables': 11730736000.0, 'inventory': 742092000.0, 'otherCurrentAssets': 202839000.0, 'nonCurrrentAssetsOther': 1275568000.0, 'propertyPlantEquipment': 1405216000.0, 'propertyPlantAndEquipmentGross': 3586488000.0, 'accumulatedDepreciation': None, 'longTermInvestments': None, 'goodWill': 2672253000.0, 'intangibleAssets': 607159000.0, 'otherAssets': 1483279000.0, 'netTangibleAssets': 3054892000.0, 'nonCurrentAssetsTotal': 11010031000.0, 'liabilitiesAndStockholdersEquity': 35664345000.0, 'totalLiab': 28636142000.0, 'totalCurrentLiabilities': 17191296000.0, 'shortTermDebt': 1927366000.0, 'shortLongTermDebtTotal': 11046150000.0, 'capitalLeaseObligations': 567770000.0, 'accountsPayable': 5940236000.0, 'otherCurrentLiab': 266697000.0, 'nonCurrentLiabilitiesTotal': 11444846000.0, 'longTermDebtTotal': 9118784000.0, 'longTermDebt': 8647691000.0, 'otherLiab': 2326062000.0, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': 228532000.0, 'totalStockholderEquity': 6334304000.0, 'commonStockTotalEquity': None, 'commonStock': 152332000.0, 'commonStockSharesOutstanding': 283680866.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': 3045413000.0, 'accumulatedOtherComprehensiveIncome': 6507509000.0, 'otherStockholderEquity': -3370950000.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEqu

In [45]:
df_balance[df_balance["comprobacion_2"] == "Error"].sort_values("comprobacion",ascending=False) 

,currency,yearly,ticker,totalPatrimonioNeto+pasivo,totalPatrimonioNeto,totalPasivo,cuentasAPagar_proveedores,efectivo,totalActivos,año,comprobacion,comprobacion_2
109,EUR,"{'totalAssets': 54947100000.0, 'totalCurrentAssets': 3648200000.0, 'cashAndShortTermInvestments': 23767900000.0, 'cash': 2086000000.0, 'shortTermInvestments': 21681900000.0, 'netReceivables': -23824300000.0, 'inventory': 56400000.0, 'otherCurrentAssets': 3648200000.0, 'nonCurrrentAssetsOther': 33029100000.0, 'propertyPlantEquipment': None, 'propertyPlantAndEquipmentGross': 2144200000.0, 'accumulatedDepreciation': None, 'longTermInvestments': 39964300000.0, 'goodWill': 1353700000.0, 'intangibleAssets': 1326900000.0, 'otherAssets': 7500300000.0, 'netTangibleAssets': None, 'nonCurrentAssetsTotal': 45360800000.0, 'liabilitiesAndStockholdersEquity': 49009000000.0, 'totalLiab': 45290800000.0, 'totalCurrentLiabilities': 34587700000.0, 'shortTermDebt': 139600000.0, 'shortLongTermDebtTotal': 3052400000.0, 'capitalLeaseObligations': 2600000.0, 'accountsPayable': 38377200000.0, 'otherCurrentLiab': -4016100000.0, 'nonCurrentLiabilitiesTotal': 4765000000.0, 'longTermDebtTotal': None, 'longTermDebt': 2601000000.0, 'otherLiab': None, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': None, 'totalStockholderEquity': 8466600000.0, 'commonStockTotalEquity': None, 'commonStock': 308000000.0, 'commonStockSharesOutstanding': 3064424175.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': 677200000.0, 'accumulatedOtherComprehensiveIncome': 6683600000.0, 'otherStockholderEquity': 797800000.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGoodwill': None, 'warrants': None, 'capitalSurpluse': None, 'date': '2023-12-31', 'filing_date': '2023-12-31', 'netDebt': 515000000.0, 'earningAssets': None, 'netWorkingCapital': None, 'netInvestedCapital': 11067600000.0}",MAP.MC,4.900900e+10,8.466600e+09,4.054240e+10,3.837720e+10,2.086000e+09,5.494710e+10,2023,5.938100e+09,Error
148,EUR,"{'totalAssets': 235172955000.0, 'totalCurrentAssets': 37138000000.0, 'cashAndShortTermInvestments': 37138000000.0, 'cash': 30960386000.0, 'shortTermInvestments': None, 'netReceivables': None, 'inventory': None, 'otherCurrentAssets': 33915509000.0, 'nonCurrrentAssetsOther': -37730000000.0, 'propertyPlantEquipment': None, 'propertyPlantAndEquipmentGross': 3930317000.0, 'accumulatedDepreciation': None, 'longTermInvestments': 32355321000.0, 'goodWill': 1018311000.0, 'intangibleAssets': 1464763000.0, 'otherAssets': 160305000000.0, 'netTangibleAssets': None, 'nonCurrentAssetsTotal': 37730000000.0, 'liabilitiesAndStockholdersEquity': 235173000000.0, 'totalLiab': 221293749000.0, 'totalCurrentLiabilities': 1138627000.0, 'shortTermDebt': 19829174000.0, 'shortLongTermDebtTotal': 25791000000.0, 'capitalLeaseObligations': None, 'accountsPayable': 1138627000.0, 'otherCurrentLiab': -1138627000.0, 'nonCurrentLiabilitiesTotal': 25791000000.0, 'longTermDebtTotal': None, 'longTermDebt': None, 'otherLiab': None, 'deferredLongTermLiab': None, 'deferredLongTermAssetCharges': None, 'nonCurrentLiabilitiesOther': None, 'totalStockholderEquity': 13844993000.0, 'commonStockTotalEquity': None, 'commonStock': 680000000.0, 'commonStockSharesOutstanding': 5403043905.0, 'additionalPaidInCapital': None, 'treasuryStock': None, 'preferredStockTotalEquity': None, 'preferredStockRedeemable': None, 'retainedEarningsTotalEquity': None, 'retainedEarnings': 7733963000.0, 'accumulatedOtherComprehensiveIncome': 5509245999.0, 'otherStockholderEquity': 11832000000.0, 'totalPermanentEquity': None, 'noncontrollingInterestInConsolidatedEntity': None, 'temporaryEquityRedeemableNoncontrollingInterests': None, 'accumulatedAmortization': None, 'negativeGo

Comprobamos que existen algunos valores en los cuales la regla contable de activo igual a pasivo + Patrimonio Neto no se cumple. Analizando la base de datos vemos que para la empresa Puig no había datos en el año 2020 y para los casos de Mapfre y Sabadell del año 2023 le faltan datos a la Api. Añadiremos en la función de la limpieza un sistema defensivo que, en el caso de que el dato no exista, lo elimine de la base de datos directamente.

Después de añadirlo vemos que solo quedan errores de cálculo de la API pero buscando en las CCAA de ambos vemos que los datos que necesitamos para el cálculo de nuestros ratios son correctos y nos falla el total pasivo + patrimonio neto que solo es útil para la comprobación. 

In [56]:
#Ahora que tenemos el balance limpio, vamos a crear una nueva función que nos calcule el ROE de cada acción y lo añada a un data frame nuevo donde guardaremos todos los ratios que calculemos.
def calculo_ROE_ratio(dataframe_balance=pd.read_csv('data_calculo_ratios/02.ROE.balance_IBEX35_limpio.csv'), dataframe_pyg=pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv')):
    """
    Función que calcula el ROE de cada acción y lo añade a un dataframe nuevo donde guardaremos todos los ratios que calculemos.
    """
    #creamos un dataframe vacío para almacenar los datos
    df_ratios1 = pd.DataFrame()
    df_ROE = pd.DataFrame()
    #a este dataframe le añadimos los datos del balance y el precio por ticker y año
    df_ratios = pd.merge(dataframe_balance, dataframe_pyg, on=["ticker","Año"], how="inner")
    #al data frame del roe le añadimos solo los datos que nos interesan para el ROE
    df_ROE = df_ratios[["ticker","Año","totalPatrimonioNeto","netIncome"]]
    #calculamos el ROE y lo añadimos al dataframe
    df_ROE["ROE"] = df_ROE["netIncome"] / df_ROE["totalPatrimonioNeto"]

    return df_ROE.to_csv('data_calculo_ratios/02.ROE_IBEX35.csv',index=False)

In [57]:
calculo_ROE_ratio()
#abrimos el csv que hemos guardado para ver los datos
df_ROE = pd.read_csv('data_calculo_ratios/02.ROE_IBEX35.csv')
df_ROE

C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\3252464486.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ROE["ROE"] = df_ROE["netIncome"] / df_ROE["totalPatrimonioNeto"]


,ticker,Año,totalPatrimonioNeto,netIncome,ROE
0,ACS.MC,2020,3.528479e+09,8.960500e+08,0.253948
1,ACS.MC,2021,6.334304e+09,-7.318700e+08,-0.115541
2,ACS.MC,2022,5.547396e+09,6.682270e+08,0.120458
3,ACS.MC,2023,5.329419e+09,7.801230e+08,0.146380
4,ACS.MC,2024,4.714213e+09,8.275800e+08,0.175550
...,...,...,...,...,...
169,UNI.MC,2020,4.004719e+09,7.783100e+07,0.019435
170,UNI.MC,2021,6.325615e+09,1.113202e+09,0.175983
171,UNI.MC,2022,6.463504e+09,2.775760e+08,0.042945
172,UNI.MC,2023,6.643626e+09,2.667030e+08,0.040144


Una vez que tenemos el ROE, sacamos nuestro siguiente ratio, el EV/EBIT. Los datos que necesitamos para obtenerlo son: 

    MarketCap: lo obtendremos multiplicando el valor de las acciones por el número de acciones en circulación
    Caja: lo tenemos en el fichero del balance
    Deuda financiera: en el fichero del balance tenemos el total pasivo y las cuentas a pagar, la deuda financiera es uno menos el otro
    Ebit: lo tenemos en la tabla del resultado

In [68]:
#Creamos una nueva función para calcular el EV/EBIT
def calculo_EV_EBIT_ratio(dataframe_balance=pd.read_csv('data_calculo_ratios/02.ROE.balance_IBEX35_limpio.csv'), dataframe_pyg=pd.read_csv('data_calculo_ratios/01.PER.cuentas_pyg_IBEX35_limpio.csv'), dataframe_precios=pd.read_csv('data_calculo_ratios/01.PER.cotizaciones_IBEX35_ultimos_precios.csv'), dataframe_acciones=pd.read_csv('data_calculo_ratios/01.04.PER.acciones_circulacion_IBEX35.csv')):
    """
    Función que calcula el EV/EBIT de cada acción y lo añade a un dataframe nuevo donde guardaremos todos los ratios que calculemos.
    """
    #creamos un dataframe vacío para almacenar los datos
    df_ratios = pd.DataFrame()
    df_EV_EBIT = pd.DataFrame()
    #a la columna "year" del dataframe de acciones en circulación le cambiamos el nombre a "Año" para poder hacer el merge con el dataframe de la cuenta de resultados
    dataframe_acciones.rename(columns={"year":"Año"}, inplace=True)
    #a este dataframe le añadimos los datos del balance y el precio por ticker y año
    df_ratios = pd.merge(dataframe_balance, dataframe_pyg, on=["ticker","Año"], how="inner")
    df_ratios = pd.merge(df_ratios, dataframe_precios, on=["ticker","Año"], how="inner")
    df_ratios = pd.merge(df_ratios, dataframe_acciones, on=["ticker","Año"], how="inner")
    #al data frame del EV/EBIT le añadimos solo los datos que nos interesan para el EV/EBIT
    df_EV_EBIT = df_ratios[["ticker","Año","EBIT","cuentasAPagar_proveedores","totalPasivo","efectivo","Ultimo Precio","Acciones Circulacion"]]
    #calculamos el EV/EBIT y lo añadimos al dataframe
    df_EV_EBIT["Market_Cap"] = df_EV_EBIT["Ultimo Precio"] * df_EV_EBIT["Acciones Circulacion"]
    df_EV_EBIT["Deuda_financiera"] = df_EV_EBIT["totalPasivo"] - df_EV_EBIT["cuentasAPagar_proveedores"]
    df_EV_EBIT["EV"] = df_EV_EBIT["Market_Cap"] + df_EV_EBIT["Deuda_financiera"] - df_EV_EBIT["efectivo"]
    df_EV_EBIT["EV/EBIT"] = df_EV_EBIT["EV"] / df_EV_EBIT["EBIT"]
    #eliminamos las columnas que no nos interesan
    df_EV_EBIT = df_EV_EBIT[["ticker","Año","Market_Cap","Deuda_financiera","EV","EBIT","EV/EBIT"]]


    return df_EV_EBIT.to_csv('data_calculo_ratios/03.EV_EBIT_IBEX35.csv',index=False)


In [69]:
calculo_EV_EBIT_ratio()
#abrimos el csv que hemos guardado para ver los datos
df_EV_EBIT = pd.read_csv('data_calculo_ratios/03.EV_EBIT_IBEX35.csv')
df_EV_EBIT


C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\1351490479.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_EV_EBIT["Market_Cap"] = df_EV_EBIT["Ultimo Precio"] * df_EV_EBIT["Acciones Circulacion"]
C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\1351490479.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_EV_EBIT["Deuda_financiera"] = df_EV_EBIT["totalPasivo"] - df_EV_EBIT["cuentasAPagar_proveedores"]
C:\Users\ganad\AppData\Local\Temp\ipykernel_11196\1351490479.py:20: SettingWithCopyWarning:

,ticker,Año,Market_Cap,Deuda_financiera,EV,EBIT,EV/EBIT
0,ACS.MC,2020,6.060735e+09,2.518596e+10,2.348122e+10,1.085166e+09,21.638370
1,ACS.MC,2021,5.648870e+09,2.338980e+10,1.809765e+10,1.196940e+08,151.199289
2,ACS.MC,2022,6.333299e+09,2.490690e+10,2.182022e+10,1.080231e+09,20.199583
3,ACS.MC,2023,9.761484e+09,2.391693e+10,2.459112e+10,1.349721e+09,18.219412
4,ACS.MC,2024,1.237251e+10,2.792434e+10,2.888320e+10,1.836656e+09,15.725971
...,...,...,...,...,...,...,...
165,UNI.MC,2020,9.003620e+08,6.130592e+10,5.553910e+10,7.044500e+07,788.403690
166,UNI.MC,2021,1.855351e+09,1.090181e+11,8.957595e+10,9.930610e+08,90.201863
167,UNI.MC,2022,2.282912e+09,9.238591e+10,9.000790e+10,3.198630e+08,281.395171
168,UNI.MC,2023,2.068487e+09,9.033204e+10,8.436273e+10,2.786230e+08,302.784508
